In [1]:
import pandas as pd
import numpy as np

In [2]:
import os
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "smartcity.settings")
import django
django.setup()

In [3]:
from app import models

In [4]:
models.Employee.objects.count()

1326

In [5]:
faults = models.Fault.objects.all()

In [6]:
faults

<QuerySet [<Fault: Fault object>, <Fault: Fault object>, <Fault: Fault object>, <Fault: Fault object>, <Fault: Fault object>, <Fault: Fault object>, <Fault: Fault object>, <Fault: Fault object>, <Fault: Fault object>, <Fault: Fault object>, <Fault: Fault object>, <Fault: Fault object>, <Fault: Fault object>, <Fault: Fault object>, <Fault: Fault object>, <Fault: Fault object>, <Fault: Fault object>, <Fault: Fault object>, <Fault: Fault object>, <Fault: Fault object>, '...(remaining elements truncated)...']>

In [7]:
emps = models.Employee.objects.all()

In [8]:
status = ['closed', 'open', 'pending']

In [9]:
reason = ['no resources', 'high risk', 'unable to verify']

In [10]:
len(models.Employee.objects.filter(specializations__contains='sewage'))

178

In [11]:
employees = models.Employee.objects.all()

In [12]:
for fault in faults:
    stat = status[np.random.randint(0, 3)]
    reas = ('', reason[np.random.randint(0, 3)])[stat == 'pending']
    try:
        employees = models.Employee.objects.filter(specializations__contains=fault.category)
        employee = employees[np.random.randint(0, len(employees))]
        models.CaseManager.objects.create(
        fault = fault,
        responder = employee,
        status = stat,
        reason = reas)
    except:
        pass

In [15]:
models.CaseManager.objects.all().count()
    

2749

In [ ]:
models.CaseManager.objects.count()

In [ ]:
!ls -l ../data/

In [ ]:
data = pd.read_excel("../data/emm_queries.xlsx", sheetname=None)

In [ ]:
water_df = pd.DataFrame(raw_data['Water problems'])
waste_df = pd.DataFrame(raw_data[ 'Waste problems'])
sewage_df = pd.DataFrame(raw_data['Sewerage problems'])
electric_df = pd.DataFrame(raw_data['Electricity problems'])
traffic_df = pd.DataFrame(raw_data['Traffic lights problems'])
street_df = pd.DataFrame(raw_data['Street Lighting Fault'])
road_df = pd.DataFrame(raw_data['Road related problems'])

In [ ]:
#dfs = ['water_df', 'waste_df', 'sewage_df', 'electric_df', 'traffic_df', 'street_df', 'road_df']
dfs = [water_df, waste_df, sewage_df, electric_df, traffic_df, street_df, road_df]

In [ ]:
waste_df.head()

In [ ]:
count = 1
water_defects = ['leaking tap', 'burst pipe', 'cutoff', 'shortages']
sewage_defects = ['leaking toilet', 'burst pipe', 'blocked']
electric_defects = ['power outage', 'mulfunctions', 'no electricity']

In [ ]:
def change_defect_col_bame(df):
    return df.rename(columns={'defect_type':'DEFECT'}, inplace=True)

In [ ]:
electric_df.head()

In [ ]:
water_df.head()

In [ ]:
waste_df.head()

In [ ]:
for df in dfs:
    df.dropna(axis=1, how='all', inplace=True)
    df.drop_duplicates(inplace=True)
    
    if count == 1:
        df['CATEGORY'] = 'WATER'
        df['DEFECT'] = None
        df.DEFECT = df.DEFECT.transform(lambda x: str(x).replace('None', water_defects[np.random.randint(0, 4)]))
    elif count == 2:
        df['CATEGORY'] = 'WASTE'
        df = change_defect_col_bame(df)
    elif count == 3:
        df['CATEGORY'] = 'SEWAGE'
        df['DEFECT'] = None
        df.DEFECT = df.DEFECT.transform(lambda x: str(x).replace('None', sewage_defects[np.random.randint(0, 3)]))
    elif count == 4:
        df['CATEGORY'] = 'ELECTRIC'
        df['DEFECT'] = None
        df.DEFECT = df.DEFECT.transform(lambda x: str(x).replace('None', electric_defects[np.random.randint(0, 3)]))
    elif count == 5:
        df['CATEGORY'] = 'TRAFFIC'
#        df.rename({'defect_type':'DEFECT'}, inplace=True)
        df = change_defect_col_bame(df)
    elif count == 6:
        df['CATEGORY'] = 'STREET'
#        df.rename({'defect_type':'DEFECT'}, inplace=True)
        df = change_defect_col_bame(df)
    else:
        df['CATEGORY'] = 'ROAD'
#        df.rename({'defect_type':'DEFECT'}, inplace=True)
        df = change_defect_col_bame(df)
    
    count += 1

In [ ]:
street_df.head()

In [ ]:
xls_writer = pd.ExcelWriter("../data/emm_quies_proc.xlsx", engine='xlsxwriter')

In [ ]:
count = 0
for df in dfs:
    df.to_excel(xls_writer, sheet_name=str(count), index=False, header=True)
    count += 1

In [ ]:
xls_writer.save()

In [ ]:
from bokeh.core.properties import value
from bokeh.io import show, output_file
from bokeh.plotting import figure

In [ ]:
output_file("visuals.html")

In [ ]:
street_df.head()

In [ ]:
pd.unique(street_df.DEFECT)

In [ ]:
street_df['month_submitted'] = street_df.DateSubmitted.transform(lambda x: str(x).split('-')[1])

In [ ]:
p = figure(x_range=pd.unique(list(street_df.month_submitted)), 
           plot_height=350, 
           title="Defect Type by Month", 
           toolbar_location=None, 
           tools="")

p.vbar_stack(list(pd.unique(street_df.DEFECT)), 
             x='fruits', 
             width=0.9, 
             source=street_df, legend=[value(x) for x in list(pd.unique(street_df.DEFECT))])

In [ ]:
p.y_range.start = 0
p.x_range.range_padding = 0.1
p.xgrid.grid_line_color = None
p.axis.minor_tick_line_color = None
p.outline_line_color = None
p.legend.location = "top_left"
p.legend.orientation = "horizontal"